In [1]:
# Configure Notebook

# enable autoreloading 
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

# import the required libraries
import plotly.graph_objs as go
import plotly.offline as py

import numpy as np
from ipywidgets import interactive, VBox, widgets, interactive_output

#py.init_notebook_mode()
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,4)

In [2]:
from pathlib import Path; Path.cwd()

WindowsPath('D:/Users/lfrison/Python/i4b/notebooks')

In [3]:
# Setup Simulation Environment

# Import the modules
import src.models.model_buildings as model_buildings
import src.models.model_hvac as model_hvac
import src.simulator as simulator
from src.disturbances import generate_disturbances
from src.controller.pid.report import plot_temperature_deviations, plot_system_performance, plot_bldg_temperatures

# Load example building data
from data.buildings.sfh_1958_1968 import sfh_1958_1968_0_soc as building

# Initialize the building model
bldg_model = model_buildings.Building(params    = building, # More example buildings can be found in data/buildings/.
                                      mdot_hp   = 0.25,     # Massflow of the heat supply system. [kg/s]
                                      method    = '4R3C',   # For available calculation methods see: model_buildings.py.
                                      )
# Initalize the heat pump model
hp_model = model_hvac.Heatpump_AW(mdot_HP = 0.25)

# Generate disturbance data for a week
p_hourly = generate_disturbances(bldg_model, repo_filepath='..')[0:168]

# Prepare the simulation
timestep = 900 # [s] simulation timestep
# resample disturbances to the simulation timestep
p = p_hourly.resample(f'{timestep}S').ffill() 

# Initialize the simulator
Simulator = simulator.Model_simulator(bldg_model = bldg_model,
                                      hp_model   = hp_model,
                                      timestep   = timestep)

# Set the initial states
state_keys = bldg_model.state_keys
x_init = {key : 20 for key in state_keys}

# set the wall temperature to the arithmetric mean of the initial temperatures of the room and ambient temperature
if 'T_wall' in state_keys:
    x_init['T_wall'] = (x_init['T_room'] + p['T_amb'][0]) / 2

x_init['T_hp_ret'] = 30

In [5]:
# Create interactive plot

from src.controller.pid.report import plot_report, plot_temperatures, plot_disturbances, plot_table_of_results

def update_graph(T_room_set, kp, ki, kd):
    res = Simulator.simulate(x_init = x_init, T_room_set = T_room_set, p = p, ctrl_method = "pid", KP = kp, KI = ki, KD = kd)
    plot_temperatures(res, p)
    #plot_disturbances(p)
    #plot_temperature_deviations(res)
    plot_system_performance(res)
    plot_table_of_results(res)

# manually determined control parameters for 2R2C
kp, ki, kd = 7.2, 1.2, 24000
# 4R3C
kp, ki, kd = 0.5, 0.9, 1300

# Create sliders
T_room_set_slider = widgets.FloatSlider(value = 20, min = 15, max = 30, step = 1, description = "T_room_set", ) # [degC]
#!ToDo: Find a way to determine good min, max and step values for the sliders
kp_slider = widgets.FloatSlider(value = kp, min = 0, max = 20, step = 0.1, description = 'kp') # [W/K]
ki_slider = widgets.FloatSlider(value = ki, min = 0, max = 2, step = 0.05, description = "ki") # [W/K]
kd_slider = widgets.FloatSlider(value = kd, min = 0, max = 50000, step = 100, description="kd") # [Ws/K]

pid_widgets = widgets.VBox([kp_slider, ki_slider, kd_slider])
input_widgets = widgets.HBox([T_room_set_slider, pid_widgets])

# Make them interactive
output = interactive_output(update_graph, {'T_room_set': T_room_set_slider, 'kp': kp_slider, 'ki': ki_slider, 'kd': kd_slider})

# Create output
vb = VBox((input_widgets, output)) # create vertical box with sliders and graph
vb.layout.align_items = 'center'
vb